<a href="https://colab.research.google.com/github/SarthakMonga2002/data/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
import pdfplumber
import os


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/redbull_model/

/content/gdrive/My Drive/redbull_model


In [ ]:
INPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/input_folder/"
OUTPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/output_text/"

In [ ]:
for fl in os.listdir(INPUT_FOLDER):
  pdf = pdfplumber.open(INPUT_FOLDER+fl)
  pdf_text=pdf.pages[0].extract_text()
  with open(OUTPUT_FOLDER + fl.replace('.pdf','.txt'),'w') as f:
    f.write(pdf_text)

In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp = spacy.blank("en")
db = DocBin()

In [ ]:
import json
f = open('data.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("Skipping entity")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./training_data.spacy") 

100%|██████████| 22/22 [00:00<00:00, 3676.43it/s]


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency     

2023-04-26 15:52:27.844734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-04-26 15:52:41.338866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2023-04-26 15:52:47,068] [INFO] Set up nlp object from config
[2023-04-26 15:52:47,079] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-26 15:52:47,082] [INFO] Created vocabulary
[2023-04-26 15:52:47,082] [INFO] Finished initializing nlp object
[2023-04-26 15:52:47,182] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     23.85    0.00    0.00    0.00    0.00
200     200

In [ ]:
nlp_ner = spacy.load("/content/gdrive/MyDrive/redbull_model/model-best")

In [ ]:
doc =nlp_ner('''Bill From
TAX INVOICE
Red Bull India Pvt Ltd.
C/O AWL India Pvt Ltd, Khasra No 393, 394,404-406, Invoice No: 4250010476
Extended Lal Dora Village Invoice Date: 30-11-2021
Alipur, Delhi Ref. No. 194
New Delhi - 110036 Ref Date: 26.11.2021
State Code: 7 GSTIN:07AADCR5842H1ZA Place of Supply: 7
Invoice To
Transporter: 07AAGCA4705P1ZF
Shiv Enterprises
TransporterName: AWL INDIA PVT LTD
6/18, 3rd Floor, Ramesh Nagar
Vehicle No: DL01LM1342
Delhi
LR no: DEL1080005663
Delhi - 110015
Mode of Transport:Road
State Code: 7 GSTIN:07AGPPG6415Q1ZS
Delivery Address
Shiv Enterprises
6/18, 3rd Floor, Ramesh Nagar
Delhi
Delhi - 110015
IRN: c4d2454ace1664443c1d0a29681ae9fc1850776baeeeb70109557a671aa3e159
Rate Per
Serial No Material Description HSN/SAC Qty Gross Amt. Discount Amt. Net Amt. GST Rate % GST Tax Amt. Total Amt. (INR)
Item
IGST: 0.00
Red Bull ED IN Alu Can 24x350ml
CGST: 2,721.00
RB201709 GST: 28
10 22029990 10 BOX 2,063.90 20,639.00 1,200.00 19,439.00 SGST/UGST: 27,214.00
Mfg. Dt.: 14.04.2021-10 CESS: 12
2,721.00
Batch No: 1915069
CESS: 2,333.00
IGST: 0.00
Red Bull IN Sugar Free 250ml WA
CGST: 2,158.00
RB205906 GST: 28
20 22029990 10 BOX 1,661.07 16,610.70 1,200.00 15,410.70 SGST/UGST: 21,575.70
Mfg. Dt.: 15.06.2021-10 CESS: 12
2,158.00
Batch No: 1929483
CESS: 1,849.00
IGST: 0.00
Red Bull Energy Drink IN 250ml
CGST: 19,417.00
RB30925 GST: 28
30 22029990 90 BOX 1,661.07 149,496.30 10,800.00 138,696.30 SGST/UGST: 194,174.30
Mfg. Dt.: 01.05.2021-90 CESS: 12
19,417.00
Batch No: 1918918
CESS: 16,644.00
Total 186,746.00 13,200.00 173,546.00 69,418.00 242,964.00
TCS 0.00
Total Amount 242,964.00
Amount in words:INR Two Lakh Forty Two Thousand Nine Hundred and Sixty Four Only
Payment Terms - Due immediately
Declaration:
No credit of additional duty of custom (ADC) is admissible to the buyer. We declare that this document shows the actual price of the goods described and that all particular are true and
correct. 
We hereby certify that food/foods mentioned in this document is / are warranted to be of the nature and quality which it/these purports/purport to be.
No. of Box: 110
Gross weight - 765.360
FSSAI Lic. No - 13314002000283
For Red Bull India Pvt. Ltd.
Authorised signatory
Red Bull India Pvt. Ltd.
Registered Office Address: 1st Floor, B Wing, 215 Atrium, Kanakia Spaces, Chakala, Andheri Kurla Road, Andheri (E) Mumbai-400093. www.redbull.in
Tel:+91-22-66180888, Fax:+91-22-66180808, Email: info.in@redbull.com, CIN: U51909MH2007FTC169580 PAN: AADCR5842H''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
import pandas as pd

In [ ]:
all_data =[]
for fl in os.listdir(INPUT_FOLDER):
    pdf_obj = pdfplumber.open(INPUT_FOLDER + fl)
    for page in pdf_obj.pages:
        tables = page.extract_tables()
        page_txt = page.extract_text()
        for table in tables:
            result=table[0]
            if 'INVOICE' in ''.join(table[0]):
                invoice_no =  table[1][0].split('\n')[0].replace('Invoice No:','').strip()
                invoice_date =  table[1][0].split('\n')[1].replace('Invoice Date:','').strip()
                transporter =  table[1][0].split('\n')[5].replace('Transporter:','').strip()
                transporter_name =  table[1][0].split('\n')[6].replace('TransporterName:','').strip()
                vehicle =  table[1][0].split('\n')[7].replace('Vehicle No:','').strip()   
            if 'Serial No' in ''.join(table[0]) or 'Item' in ''.join(table[0]):
                table_df = pd.DataFrame(table[1::],columns=table[0])
                for idx, row in table_df.iterrows():
                    if row['Serial No'].isnumeric():
                        if idx == 0:
                            doc = nlp_ner(page_txt)
                            all_label_text = []
                            for ent in doc.ents:
                              if ent.label_ == "COMPANY NAME":
                                company_name = ent.text
                              if ent.label_ == "GSTIN NUMBER":
                                company_gstin = ent.text
                              if ent.label_ == "VENDOR NAME":
                                vehicle_no = ent.text
                              if ent.label_ == "BUYER":
                                buyer_gstin = ent.text
                              if ent.label_ == "GROSS AMT.":
                                amt = ent.text  
                            tot_row = table_df[table_df['Serial No'] == 'Total']
                            all_data.append([invoice_no,invoice_date,transporter,transporter_name,vehicle,company_gstin,row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty']])
                        else:
                            all_data.append(['','','','','','',row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty']])                                             

In [ ]:
out_df = pd.DataFrame(all_data, columns= ['Invoice No','INVOICE DATE','Transporter ID','COMPANY NAME','VEHICLE NUMBER','COMPANY PAN NUMBER','Serial No', 'Material Description', 'HSN/SAC', 'Qty'])

In [ ]:
out_df

,Invoice No,INVOICE DATE,Transporter ID,COMPANY NAME,VEHICLE NUMBER,COMPANY PAN NUMBER,Serial No,Material Description,HSN/SAC,Qty
0,4203304685,06-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UP32PN4330,PAN: AADCR5842H,10,Red Bull ED IN Alu Can 24x250ml\nWA\nRB226600\...,22029990,100 BOX
1,4203304676,04-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UP30A9433,PAN: AADCR5842H,10,Red Bull ED IN Alu Can 24x350ml\nRB201709\nMfg...,22029990,10 BOX
2,,,,,,,20,Red Bull ED IN Alu Can 24x250ml\nWA\nRB226600\...,22029990,730 BOX
3,4203304671,04-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UP16HT1374,PAN: AADCR5842H,10,Red Bull ED IN Alu Can 24x330ml\nWA\nRB217737\...,22029990,40 BOX
4,,,,,,,20,Red Bull IN Sugar Free 250ml WA\nRB205906\nMfg...,22029990,5 BOX
5,,,,,,,30,Red Bull ED IN Alu Can 24x250ml\nWA\nRB226600\...,22029990,255 BOX
6,4203304670,04-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UP35T6021,PAN: AADCR5842H,10,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,200 BOX
7,4203304682,06-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UK06CB3061,PAN: AADCR5842H,10,Red Bull ED IN Alu Can 24x250ml\nWA\nRB226600\...,22029990,299 BOX
8,4203304679,06-12-2021,09AAGCA4705P1ZB,AWL INDIA PVT LTD,UP14GT2103,PAN: AADCR5842H,10,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,35 BOX
9,,,,,,,20,RB ED IN 250ml 4P PMC Big\nMoment\nRB236559\nM...,22029990,125 BOX


In [ ]:
out_df.to_csv(OUTPUT_FOLDER+'DATA_AUTOMATION.csv', sep=self.delimiter, float_format='%.2f',index=False, line_terminator='\n')